# Data transformation

Dans cette partie nous allons formaer la table que nous allons utiliser pour nos algorithmes.
Dans un premier temps nous choississons comme objectif de prédire les recettes d'un film.
On va donc prendre en compte toutes les données relatives au travail en amont de la sortie et donc sortir les donnnées issues des utilisateurs comme par exemple la popularité.
Dans un deuxième temps on va essayer de tester une hypothèse sur ces revenues :
Est-ce que la diversité (sexe, nationnalité) influence ces revenus ?

In [ ]:
# Libraries import
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter

In [ ]:
# Dataset import

credits = pd.read_csv('./archive/credits.csv', delimiter=',',index_col="id")
# keywords = pd.read_csv('./archive/keywords.csv', delimiter=',',index_col="id") # Pas très informatif pour notre question
movies = pd.read_csv('./archive/movies_metadata.csv', delimiter=',').\
                     drop(['belongs_to_collection', 'homepage', 'imdb_id', 'poster_path', 'status', 'title', 'video'], axis=1).\
                     drop([19730, 29503, 35587]) 

In [ ]:
movies['id'] = movies['id'].astype('int64') # incorrect datatype for merge 
df = movies.merge(credits, on='id')

In [ ]:
# Converssion Json format en nominal format
def get_dictionary(s):
    try:
        d = eval(s)
    except:
        d = {}
    return d

Pour obtenir cette notion de diversité, on peut ce baser sur le genre des personnes et leur ordre d'importance.
Le genre est associé au personnage et nom à l'acteur. (2 = homme, 1 = femme)
Pour l'équipe de tournage, on a accès au genre mais connaitre la position dans l'organigramme et donc son influence est plus compliqué.

In [ ]:
# df.keywords = df.keywords.map(lambda x: [d['name'] for d in get_dictionary(x)]).map(lambda x: ','.join(map(str, x)))
df.genres = df.genres.map(lambda x: [d['name'] for d in get_dictionary(x)]).map(lambda x: ','.join(map(str, x)))
# df.production_companies = df.production_companies.map(lambda x: [d['name'] for d in get_dictionary(x)]).map(lambda x: ','.join(map(str, x))) # trop de valeur manquantes
df.spoken_languages = df.spoken_languages.map(lambda x: [d['name'] for d in get_dictionary(x)]).map(lambda x: ','.join(map(str, x)))
# df.production_countries = df.production_countries.map(lambda x: [d['name'] for d in get_dictionary(x)]).map(lambda x: ','.join(map(str, x)))

# New columns
# df['characters'] = df.cast.map(lambda x: [d['character'] for d in get_dictionary(x)]).map(lambda x: ','.join(map(str, x)))
df['characters_gender'] = df.cast.map(lambda x: [d['gender'] for d in get_dictionary(x)]).map(lambda x: ','.join(map(str, x)))
# df['actors'] = df.cast.map(lambda x: [d['name'] for d in get_dictionary(x)]).map(lambda x: ','.join(map(str, x)))
df['actors_order'] = df.cast.map(lambda x: [d['order'] for d in get_dictionary(x)]).map(lambda x: ','.join(map(str, x)))
# df.crew = df.crew.map(lambda x: [d['name'] for d in get_dictionary(x)]).map(lambda x: ','.join(map(str, x)))
df['crew_gender'] = df.cast.map(lambda x: [d['gender'] for d in get_dictionary(x)]).map(lambda x: ','.join(map(str, x)))


In [ ]:
df.pop('production_companies')
df.pop('production_countries')
df.pop('cast')
df.pop('crew')
df.pop('original_title')
df.pop('popularity')
df.pop('tagline')
df.pop('vote_average')
df.pop('vote_count')
df.pop('overview')
df.pop('id')

In [ ]:
df.head()

In [ ]:
language_list = [i for i in df.original_language]
print(Counter(language_list))
print(len(Counter(language_list))) 

Il y a trop de langues pour que cela soit pertinant à transcrire en onehotencoding. On peut cependant noter que la plupart des langues sont peu présentes.
On va donc réduire l'ensemble des langues utilisés à 'en' et 'other'. Si la puissance de calcul est suifisant on pourrait voir à rajouter d'autres langues.

0 = other,
1 = english

Les valeurs manquantes seront remplacé par de l'anglais (valeur la plus probable)

In [ ]:
df.original_language = df.original_language.fillna(1)
df.original_language = df.original_language.replace(to_replace=r'(^((?!en).)*$)', value=0, regex=True)
df.original_language = df.original_language.replace(to_replace=r'en', value=1, regex=True)

In [ ]:
genres_list = []
for i in df['genres']:
    genres_list.extend(i.split(','))
print(Counter(genres_list))
print(len(Counter(genres_list))) # 21 genres possibilité de faire du onehotencoding mais rajout de beaucoup de colonnes

Il y a 21 genres de film. Comme vu dans le notebook de visualisation 50% des films sont dans [drama, comedy, thriller, romance].
Néanmoins, il n'y a pas de marche significative dans les genres après pour pouvoir simplifier les colonnes.
Pour le moment on ne va pas réduire le nombre de colonnes.

In [ ]:
# companies_list = [i for i in df.production_companies]
# print(Counter(companies_list))
# print(len(Counter(companies_list))) 
# Beaucoup de valeur manquantes 

Beaucoup de valeurs manquantes (12000) donc on va supprimer cette colonne.

In [ ]:
companies_list = [i for i in df.production_countries]
# print(Counter(companies_list))
print(len(Counter(companies_list))) 
# Beaucoup de valeur manquantes mais large

Beaucoup de valeur manquantes (6500). Dans la mesure où on a déjà une information quant à la langue d'origine. On peut supposer que cette colonne ajoute que peut d'informations supplémentaires (surtout si on a décidé de séparer en 'english' et 'other'). On ne va donc pas garder cette colonne. Même si il y est hautement probable que le pays d'origine influence le revenu.

In [ ]:
language_list = []
for i in df['spoken_languages']:
    language_list.extend(i.split(','))
print(Counter(language_list))
print(len(Counter(language_list)))

Il y a 75 langues différentes. On va procéder au même tri que précédemment, i.e les films en anglais versus les autres.

In [ ]:
df.spoken_languages = df.spoken_languages.fillna(1)
df.spoken_languages = df.spoken_languages.replace(to_replace=r'(^((?!English).)*$)', value=0, regex=True)
df.spoken_languages = df.spoken_languages.replace(to_replace=r'English', value=1, regex=True)

### Characters

On va garder le nombre de characters dans le film faire puis garder le pourcentage de 0,1,2.


In [ ]:
gender_list = []
for i in df['characters_gender']:
    gender_list.extend(i.split(','))
print(Counter(gender_list))
print(len(Counter(gender_list)))

Il y a quelques valeurs manqunates que nous allons ajouter à 0 (correspondant à 'innconu')

In [ ]:
df.characters_gender = df.characters_gender.fillna(0)